# 캠페인 및 필터 배포<a class="anchor" id="top"></a>

이 노트북에서는 Amazon Personalize에서 캠페인을 배포하고 상호 작용합니다.

1. [소개](#intro)
1. [캠페인 생성](#create)
1. [캠페인과 상호 작용](#interact)
1. [배치 추천](#batch)
1. [마무리](#wrapup)

## 소개 <a class="anchor" id="intro"></a>
[맨 위로 이동](#top)

이 시점에는 몇 개의 솔루션과 각 솔루션의 버전이 하나 만들어져 있습니다. 일단 솔루션 버전이 생성되면 해당 버전으로부터 추천을 받고 전반적인 동작을 파악할 수 있습니다.

이 노트북에서는 이전 노트북의 각 솔루션 버전을 개별 캠페인에 배포하는 것부터 시작합니다. 캠페인이 활성화되면 추천을 쿼리하기 위한 리소스와 사용자에게 좀 더 읽기 쉬운 출력으로 요약하는 헬퍼 함수가 제공됩니다.

Amazon Personalize를 사용하는 고객과 마찬가지로, 헬퍼 함수를 데이터 입력 파일의 구조에 맞게 수정하여 추가 렌더링이 계속 작동하도록 할 수 있습니다.

시작하려면 라이브러리를 가져오고 이전 노트북의 값을 로드하고 SDK를 로드해야 합니다.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## 캠페인 생성 <a class="anchor" id="create"></a>
[맨 위로 이동](#top)

캠페인은 호스팅된 솔루션 버전으로, 추천을 쿼리할 수 있는 엔드포인트입니다. 요금은 처리 용량(초당 사용자의 개인화 요청)을 추정하여 정해집니다. 캠페인을 배포할 때 최소 초당 처리량(TPS) 값을 설정합니다. 이 서비스는 AWS 내의 다른 서비스와 마찬가지로 수요에 따라 자동으로 확장되지만, 대기 시간이 중요한 경우에는 더 많은 수요에 대비해 미리 프로비저닝하는 것이 좋습니다. 이 POC 및 데모에서는 모든 최소 처리량 임계값을 1로 설정합니다. 자세한 내용은 [요금 페이지](https://aws.amazon.com/personalize/pricing/)를 참조하세요.

이제 캠페인을 구축해 보겠습니다.

### 사용자 개인화

사용자 개인화 솔루션 버전에 대한 캠페인을 배포합니다. 캠페인을 배포하는 데 10분 정도 걸릴 수 있습니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 여러 캠페인을 만드는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 캠페인에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [ ]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

### SIMS

사용 중인 SIMS 솔루션 버전에 대한 캠페인을 배포합니다. 캠페인을 배포하는 데 10분 정도 걸릴 수 있습니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 여러 캠페인을 만드는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 캠페인에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [ ]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

### 개인별 순위

개인화된 순위 솔루션 버전에 대한 캠페인을 배포합니다. 캠페인을 배포하는 데 10분 정도 걸릴 수 있습니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 여러 캠페인을 만드는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 캠페인에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [ ]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

### 캠페인 생성 상태 보기

예고해 드린 대로, 콘솔에서 상태 업데이트를 보는 방법은 다음과 같습니다.

* 다른 브라우저 탭에서 이 노트북 인스턴스를 열 때 AWS 콘솔이 이미 열려 있어야 합니다.
* 해당 탭으로 전환하고 맨 위에서 `Personalize` 서비스를 검색한 다음 해당 서비스 페이지로 이동합니다.
* `View dataset groups`를 클릭합니다.
* 데이터 세트 그룹의 이름을 클릭합니다. 이름에 POC가 있을 가능성이 높습니다.
* `Campaigns`를 클릭합니다.
* 이제 캠페인 상태가 표시된 열을 포함하여, 위에서 생성한 모든 캠페인의 목록이 표시됩니다. `Active` 상태가 되면 캠페인을 쿼리할 수 있습니다.

또는 아래의 셀을 실행하여 캠페인 생성 상태를 추적합니다.

In [ ]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

## 필터 생성 <a class="anchor" id="interact"></a>
[맨 위로 이동](#top)

이제 모든 캠페인이 배포되고 활성화되었으므로 필터를 생성할 수 있습니다. 필터는 항목과 이벤트 모두에 대해 만들 수 있습니다. 온디맨드 비디오 필터의 일반적인 사용 사례는 다음과 같습니다.

항목 메타데이터를 기반으로 하는 범주형 필터 - 항목 메타데이터에는 장르, 키워드, 연도, 연대 등, 타이틀에 대한 정보가 포함되어 있는 경우가 많습니다. 이 정보를 필터링하면 해당 데이터 내에서 액션 영화와 같은 추천을 제공할 수 있습니다.

이벤트 - 특정 이벤트를 필터링하고 해당 이벤트를 기준으로 결과를 제공할 수 있습니다. 예를 들어 추천을 "시청 제안" 추천의 타이틀을 "다시 시청" 추천으로 옮길 수 있습니다.

항목 메타데이터 및 사용자 상호 작용에 대해 알아보겠습니다. 이를 통해 어떤 유형의 필터를 만들 수 있는지 알 수 있습니다.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)
#interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
items_df.head(10)
#interactions_df.head(10)

이제 필터링할 장르를 결정해야 합니다. 여기서는 모든 장르의 목록이 필요합니다. 먼저 GENRE 열의 모든 고유 값을 가져온 후 `|`(있는 경우)에서 문자열을 분할합니다. 그러면 모두 긴 목록에 추가되고, 이 목록은 효율성을 높이기 위해 세트로 변환됩니다. 그런 다음 해당 세트가 반복 가능한 목록으로 만들어지고, 그러면 필터 생성 API를 사용할 수 있습니다.

In [ ]:
unique_genre_field_values = items_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [ ]:
genres_to_filter

이를 통해 데이터 세트에 존재하는 모든 장르를 포함할 수 있습니다. 현재 Personalize에서 필터는 총 10개로 제한됩니다. 장르가 더 많기 때문에 무작위로 7개를 선택하여 나중에 2개의 상호 작용 기반 필터와 연도를 기준으로 한 추천을 위한 추가 필터를 사용할 수 있도록 여유분을 확보합니다.

In [ ]:
genres_to_filter = random.sample(genres_to_filter, 7)
genres_to_filter

이제 메타데이터 장르 필터에 대한 목록을 만든 다음 아래의 셀을 실행하여 실제 필터를 만듭니다. 이 작업을 완료하는 데 몇 분 정도 걸립니다.

In [ ]:
# Create a list for the filters:
meta_filter_arns = []

In [ ]:
# Iterate through Genres
for genre in genres_to_filter:
    # Start by creating a filter
    try:
        createfilter_response = personalize.create_filter(
            name=genre,
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
        )
        # Add the ARN to the list
        meta_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=genre,
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
            )
            # Add the ARN to the list
            meta_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

시청한 콘텐츠와 시청하지 않은 콘텐츠에 대한 2개의 이벤트 필터도 생성합니다.

In [ ]:
# Create a dataframe for the interactions by reading in the correct source CSV
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
interactions_df.head(10)

시청한 콘텐츠와 시청하지 않은 콘텐츠에 대한 2개의 이벤트 필터도 생성합니다.

In [ ]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )


마지막으로 항목 메타데이터에서 연도를 사용할 수 있으므로, 특정 연대에 개봉한 영화만 추천하는 연대 필터를 만들어 보겠습니다. 이 워크숍에서는 1970년대 영화를 선택합니다. 

In [ ]:
createdecadefilter_response = personalize.create_filter(name='1970s',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980'
    )

작업을 완료하기 전에, 나중에 사용할 수 있도록 이러한 필터를 목록에 추가해야 합니다.

In [ ]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

In [ ]:
decade_filter_arns = [createdecadefilter_response['filterArn']]

In [ ]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store meta_filter_arns
%store interaction_filter_arns
%store decade_filter_arns


이제 마지막 탐색 노트북 `05_Interacting_with_Campaigns_and_Filters.ipynb`로 넘어가겠습니다. 브라우저에서 이 노트북을 열면 캠페인과의 상호 작용을 시작하고 추천을 받을 수 있습니다.